In [68]:
import pandas as pd
import numpy as np
import torch as pt

import sklearn as sk
from sklearn import preprocessing
import tensorflow as tf

import matplotlib.pyplot as plot

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [83]:
climate_data = pd.read_csv('Data_climate_indices_V2A.csv', index_col=0)

# Finding the overall training and testing data.

training = climate_data[1:361]
testing = climate_data[361:len(climate_data)]

# Splitting the x and y values for training and testing.

train_x = training[training.columns[1:training.shape[1]-1]]
train_y = training[training.columns[training.shape[1] -1:]].values[1:]

test_x = testing[testing.columns[1:testing.shape[1]-1]]
test_y = testing[testing.columns[testing.shape[1] -1:]].values[1:]

# Normalizing the training and testing variables.

min_max_scaler = preprocessing.MinMaxScaler()
train_x_scaled = min_max_scaler.fit_transform(train_x)
test_x_scaled = min_max_scaler.fit_transform(test_x)



In [80]:
# Creating the data with shifts for training.

logs = 15
shifted_x_train = []
shifted_y_train = []

for index in range(logs,train_x.shape[0] - 1):
    shifted_x_train.append(train_x_scaled[index - logs : index])
    shifted_y_train.append(train_y[index][0])

shifted_x_train, shifted_y_train = np.array(shifted_x_train), np.array(shifted_y_train)

print(shifted_x_train.shape)
print(shifted_y_train.shape)

(344, 15, 43)
(344,)


In [86]:
learning_model = Sequential()
learning_model.add(LSTM(units = 50, return_sequences = True, input_shape=(logs,train_x.shape[1])))
learning_model.add(Dropout(0.2))
learning_model.add(LSTM(units = 50))
learning_model.add(Dropout(0.2))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))




learning_model.compile(optimizer = 'adam' , loss = 'mean_squared_error', metrics='mean_absolute_percentage_error' )

In [87]:
learning_model.fit(shifted_x_train, shifted_y_train , epochs = 600 , batch_size = 12)

Epoch 1/600
29/29 [==============================] - 18s 37ms/step - loss: 12249.0234 - mean_absolute_percentage_error: 523.5484
Epoch 2/600
29/29 [==============================] - 1s 29ms/step - loss: 9674.7617 - mean_absolute_percentage_error: 1819.1016
Epoch 3/600
29/29 [==============================] - 1s 27ms/step - loss: 8812.2314 - mean_absolute_percentage_error: 2911.8945
Epoch 4/600
29/29 [==============================] - 1s 28ms/step - loss: 8421.9033 - mean_absolute_percentage_error: 2836.1565
Epoch 5/600
29/29 [==============================] - 1s 33ms/step - loss: 8331.1797 - mean_absolute_percentage_error: 3234.0552
Epoch 6/600
29/29 [==============================] - 1s 23ms/step - loss: 8061.5073 - mean_absolute_percentage_error: 3326.9309
Epoch 7/600
29/29 [==============================] - 1s 24ms/step - loss: 8031.8228 - mean_absolute_percentage_error: 3535.9854
Epoch 8/600
29/29 [==============================] - 1s 23ms/step - loss: 7946.6387 - mean_absolute_per

In [88]:
# Data shifts for the testing data.
shifted_x_test = []
for index in range(logs,test_x.shape[0]-1):
    shifted_x_test.append(test_x_scaled[index - logs : index])
shifted_x_test = np.array(shifted_x_test)

print(shifted_x_test.shape)
    

(91, 15, 43)


In [89]:
# Testing the model
# y_test = learning_model(shifted_x_test)

# print(y_test)
test_yAdjusted = test_y[0:len(test_y)-logs]
learning_model.evaluate(x=shifted_x_test, y=test_yAdjusted)
predict = learning_model.predict(shifted_x_test)

3/3 [==============================] - 4s 31ms/step


In [75]:
import seaborn as sns
sns.scatterplot(data=shifted_x_test, x="date", y="precipitation")

c:\Python38\lib\site-packages\seaborn\_oldcore.py:917: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  val in data


ValueError: Could not interpret value `date` for parameter `x`